# Load Functions

## 0. imports

In [2]:
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.utils import ImageReader
import os
import mne
import json

from utils import *
from eeg_qc import compute_eeg_pipeline, test_eeg_pipeline
from ecg_qc import ecg_qc 
from eda_qc import eda_qc
from rsp_qc import *
from mic_qc import *
from lsl_problem import *
from et_qc import *
from webcam_qc import *
import matplotlib

In [3]:
# subject = "P5415639"
# xdf_filename = f'/Users/bryan.gonzalez/CUNY_subs/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'

subject = "P5182010"
xdf_filename = f'/Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'

#xdf_filename = #Apurva's xdf file

video_filename = '/'.join(xdf_filename.split('/')[:-1])+ f'/sub-{subject}_task-CUNY_run-001_video.avi'


In [4]:
stim_df = import_stim_data(xdf_filename)

# Get Metrics

## 1. EEG

In [5]:
#Compute the eeg vars
eeg_vars, raw_cleaned, ica, eeg_df = compute_eeg_pipeline(xdf_filename, 
                                                          stim_df=stim_df, 
                                                          task='RestingState')


Opening raw data file /Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-P5182010/sub-P5182010_ses-S001_task-CUNY_run-001_eeg_clean.fif...
    Range : 0 ... 299989 =      0.000 ...   300.000 secs
Ready.
Reading 0 ... 299989  =      0.000 ...   300.000 secs...
Bad channels before robust reference: ['E8', 'E127', 'E7', 'E122', 'E73', 'E126', 'E116', 'E14', 'E95', 'Cz']
Interpolated channels: ['E8', 'E122', 'E21', 'E73', 'E116', 'E14', 'E95', 'Cz']
Bad channels after interpolation: ['E126', 'E39', 'E127', 'E117']
Using EOG channels: E25, E8
EOG channel index for this subject is: [24  7]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel E25 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandw

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 95 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 95.00
- Lower transition bandwidth: 23.75 Hz (-6 dB cutoff frequency: 83.12 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 139 samples (0.139 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    0.7s finished


Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1651 samples (1.651 s)

Percent Good Data: 78.65%
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 6601 samples (6.601 s)



...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.3s


Fitting ICA to data using 126 channels (please be patient, this may take a while)
Omitting 46532 of 299988 (15.51%) samples, retaining 253456 (84.49%) samples.


[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    0.5s finished


Selecting by explained variance: 21 components
Fitting ICA took 27.4s.


### Manual Artifact Removal

In [ ]:
fig = ica.plot_components( title='ICA Components')
# Save the ICA plot

fig[0].savefig(f'report_images/sub-{subject}_eeg_ica_components.png', bbox_inches='tight')

In [ ]:
ica.plot_properties(raw_cleaned, picks=[0,1,2,3,4]) # This exact component number probably won't work if you recompute ICA


In [ ]:
ica.plot_properties(raw_cleaned, picks=[0,1,2,3,4]) # This exact component number probably won't work if you recompute ICA


In [ ]:
ica.exclude = [0,1,2,3,4] # these are the components that we want to exclude
ica.apply(raw_cleaned)

In [ ]:
eeg_vars['components_excluded'] = ica.exclude

### Generate figures

In [ ]:
raw_cleaned.annotations.delete([i for i, desc in enumerate(raw_cleaned.annotations.description) if desc == 'blink' or desc == 'BAD_muscle'])
fig = raw_cleaned.plot(show_scrollbars=False,
                        show_scalebars=False,events=None, start=0, 
                        duration=200,n_channels=50, scalings=.35e-4, color='k', title='EEG Data after ICA')

fig.savefig(f'report_images/{subject}_eeg_cleaned.png', dpi=300, bbox_inches='tight')


fig = raw_cleaned.plot_psd(fmax=50, average=False, show=True)
fig.savefig(f'report_images/{subject}_eeg_cleaned_psd.png', dpi=300, bbox_inches='tight')

In [12]:
raw_cleaned_pathname = '/'.join(xdf_filename.split('/')[:-1]) + f'/sub-{subject}_ses-S001_task-CUNY_run-001_eeg_clean.fif'
raw_cleaned.save(raw_cleaned_pathname, overwrite=True)


eeg_print = {
    'Bad channels before robust reference': eeg_vars['bad_channels_before'], 
    'Interpolated channels': eeg_vars['interpolated_channels'], 
    'Bad channels after interpolation': eeg_vars['bad_channels_after'], 
    'Percent Good before artifact removal': f"{eeg_vars['percent_good']:.4}%",
    'Artifactual Components Excluded': ica.exclude
}

## 2. ECG

In [ ]:
[ecg_vars, ecg_plt, ps_df] = ecg_qc(xdf_filename = xdf_filename, stim_df = stim_df, task='RestingState')


In [ ]:
ecg_vars.keys()

In [ ]:
ecg_print = {
    "Effective sampling rate": f'{ecg_vars["sampling_rate"]:.4f} Hz', 
    "Signal to Noise Ratio": f'{ecg_vars["SNR"]:.4f} dB',
    "Average heart rate": f'{ecg_vars["average_heart_rate"]:.4f} bpm',
    "Kurtosis signal quality index (kSQI)": f'{ecg_vars["kurtosis_SQI"]:.4f}',
    "Power spectrum distribution (pSQI)": f'{ecg_vars["power_spectrum_distribution_SQI"]:.4f} mV²/Hz',
    "Relative power in baseline (basSQI)": f'{ecg_vars["relative_baseline_power_sqi"]:.4f}%'
}

In [ ]:
ecg_print

## 3. EDA

In [ ]:
[eda_vars, eda_plt1, eda_plt2, ps_df] = eda_qc(xdf_filename = xdf_filename, stim_df = stim_df, task= 'RestingState')

In [ ]:
eda_vars.keys()

In [ ]:
eda_print = {
    "Effective sampling rate": f'{eda_vars["sampling_rate"]:.4f} Hz', 
    "Signal to noise ratio": f'{eda_vars["snr"]:.4f} dB',
    "Signal integrity check": f'{eda_vars["signal_integrity_check"]:.4f}%',
    "Average skin conductance level": f'{eda_vars["average_scl"]:.4f} mS',
    "Skin conductance level std": f'{eda_vars["scl_sd"]:.4f} mS',
    "Skin conductance level coefficient of variation": f'{eda_vars["scl_cv"]:.4f}%',
    "Average amplitude of skin conductance response": f'{eda_vars["average_scr_amplitude"]:.4f} mS',
    "Skin conductance response validity": f'{eda_vars["sc_validity"]:.4f} %' # this will need to be changed to [scr_validity]
}

## 4. RSP

In [ ]:
rsp_vars, ps_df = rsp_qc(xdf_filename=xdf_filename, stim_df=stim_df, task='RestingState')

In [ ]:
rsp_print = {
    "Effective sampling rate": f'{rsp_vars["sampling_rate"]:.4f} Hz', 
    "Signal to noise ratio": f'{rsp_vars["rsp_snr"]:.4f} dB',
    "Breath amplitude mean": f'{rsp_vars["breath_amplitude_mean"]:.4f} V',
    "Breath amplitude std": f'{rsp_vars["breath_amplitude_std"]:.4f} V',
    "Breath amplitude range": f'{rsp_vars["breath_amplitude_range"]} V',
    "Respiration rate mean": f'{rsp_vars["rsp_rate_mean"]:.4f} bpm',
    "Respiration rate std": f'{rsp_vars["rsp_rate_std"]:.4f} bpm', 
    "Respiration rate range": f'{rsp_vars["rsp_rate_range"]} bpm', 
    "Peak to peak interval mean": f'{rsp_vars["ptp_mean"]:.4f} sec', 
    "Peak to peak interval std": f'{rsp_vars["ptp_std"]:.4f} sec', 
    "Peak to peak interval range": f'{rsp_vars["ptp_range"]} sec', 
    "Baseline drift": f'{rsp_vars["baseline_drift"]:.4f} V', 
    "Autocorrelation at typical breath cycle": f'{rsp_vars["autocorrelation"]:.4f}'
}

In [ ]:
rsp_print

## 5. Mic

In [ ]:
mic_vars, mic_df = mic_qc(xdf_filename=xdf_filename, stim_df=stim_df)

In [ ]:
mic_print = {
    "Effective sampling rate": f'{mic_vars["sampling_rate"]:.4f} Hz', 
    "Difference between .wav file and lsl timestamps durations": f'{mic_vars["lsl_wav_duration_diff"]:.4f} sec', 
    "Number of NaN's": f'{mic_vars["num_NaN"]}',
    "Percent of NaN's": f'{mic_vars["percent_NaN"]:.4f}%',
    "Mic samples first quartile": f'{mic_vars["quan25"]:.4f}',
    "Mic samples third quartile": f'{mic_vars["quan75"]:.4f}',
    "Mic samples std": f'{mic_vars["std"]:.4f}',
    "Mic samples min": f'{mic_vars["min"]:.4f}',
    "Mic samples max": f'{mic_vars["max"]:.4f}'
}

## 6. Webcam

In [ ]:
webcam_vars, cam_df = webcam_qc(xdf_filename=xdf_filename,
                                stim_df=stim_df,
                                video_file=video_filename, task='RestingState')

In [ ]:
webcam_print = {
    "Effective sampling rate": f'{webcam_vars["sampling_rate"]:.4f} Hz', 
    "Collected full resting state": webcam_vars["collected_full_RestingState"], 
    "Percent of frames with face detected": f'{webcam_vars["face_perc"]:.4%}'
}

In [ ]:
webcam_print

## 7. ET

In [ ]:
et_vars, et_df = et_qc(xdf_filename = xdf_filename, stim_df = stim_df, task='RestingState')

In [ ]:
et_print = {
    "Effective sampling rate": f'{et_vars["sampling_rate"]:.4f} Hz', 
    "Flag: all coordinates have the same % validity within each measure (LR, gaze point/origin/diameter)": et_vars["flag1"], 
    "Flag: % of NaNs is the same between coordinate systems (UCS and TBCS (gaze origin) and between UCS and display area (gaze point))": et_vars["flag2"],
    "Mean difference in percent valid data between right and left eyes": f'{et_vars["LR_mean_diff"]:.4f}%',
    "Percent of data with gaze point differences of over 0.2 mm": f'{et_vars["percent_over02"]:.4f}%'
}

## Stream Durations 

### functions are in utils.py (but should be called in the report)

In [ ]:
df_map = {
    'et': et_df,
    'ps': ps_df,
    'mic': mic_df,
    'cam': cam_df
    }
    # 'eeg': eeg_df

In [ ]:
duration_vars = {"Durations of each modality + comparison to expected duration:": 
    get_durations(xdf_path=xdf_filename, task='Experiment', df_map = df_map, stim_df = stim_df)}

In [ ]:
duration_print = duration_vars

In [ ]:
# i wont run these but they are here for reference
# get_durations('RestingState', xdf_filename)
# get_durations('CampFriend', xdf_filename)
# get_durations('SocialTask', xdf_filename)
# whole_durations(xdf_filename)

## LSL Problem

In [ ]:
lsl_vars = lsl_problem_qc(xdf_filename, df_map=df_map, stim_df=stim_df, modality_to_plot='et')

In [ ]:
lsl_print = {
    "Percent of missing data in entire experiment": lsl_vars["percent_loss"],
    "Percent of missing data before social task offset": lsl_vars["loss_before_social_task"]
}

# Second Report with formatting

In [ ]:
# Modalities and corresponding data
metric_names = ["EEG", "ECG", "EDA", "RSP", "MIC","ET", "WEBCAM", "Stream Durations", "LSL"]
metrics_list = [eeg_print, ecg_print, eda_print, rsp_print, mic_print, et_print, webcam_print, duration_print, lsl_print]
# metric_names = ["ECG", "EDA","WEBCAM"]
# metrics_list = [ecg_print, eda_print,  webcam_print]

# PDF structure
parent_folder = xdf_filename.split('mobi')[0]
pdf_path = f"{parent_folder}QCReport.pdf"
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

# Define subtitle style if not already done
subtitle_style = ParagraphStyle(
    name="Subtitle",
    parent=styles["Heading2"],
    fontSize=14,
    leading=16,
    textColor="gray",
    spaceAfter=12,
    alignment=1  # Centered
)

# page number function
def add_page_number(canvas, doc):
    page_num = f'{canvas.getPageNumber()}'
    canvas.setFont("Helvetica", 9)
    canvas.drawRightString(570, 20, page_num)  # (x, y) from bottom-left


elements.append(Paragraph(f"Subject Report: {subject}", styles["Title"]))
elements.append(Paragraph(f"Collection Date: {get_collection_date(xdf_filename)}", subtitle_style))
elements.append(Spacer(1, 12))

# Format each metrics dict
for name, metrics in zip(metric_names, metrics_list):
    elements.append(Paragraph(name, styles["Heading2"]))
    for k, v in metrics.items():
        if isinstance(v, pd.DataFrame):
            data = [v.columns.tolist()] + v.values.tolist()  # Include headers
            table = Table(data, repeatRows=1)
            table.hAlign = 'LEFT'

            table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
                ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 10),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ]))
            elements.append(Paragraph(k, styles['Normal']))
            elements.append(Spacer(1, 12))

            elements.append(table)
            elements.append(Spacer(1, 12))
        else:
            text = f"<b>{k}:</b> {v}" if isinstance(v, float) else f"<b>{k}:</b> {v}"
            elements.append(Paragraph(text, styles["Normal"]))
    elements.append(Spacer(1, 12))

    # images
    folder = "report_images"
    image_keyword = name.lower()
    if os.path.exists(folder):
        for fname in sorted(os.listdir(folder)):
            if image_keyword in fname.lower() and subject in fname:
                image_path = os.path.join(folder, fname)

                # get dimensions
                image_reader = ImageReader(image_path)
                orig_width, orig_height = image_reader.getSize()
                print(orig_width, orig_height)
                if orig_width > 4000:
                    img = Image(image_path, width=orig_width/9, height=orig_height/9)
                elif orig_width > 3000:
                    img = Image(image_path, width=orig_width/7, height=orig_height/7)
                elif orig_width > 2000:
                    img = Image(image_path, width = orig_width/5, height = orig_height/5)
                elif orig_width > 1400:
                    img = Image(image_path, width=orig_width/3, height=orig_height/3)
                else:
                    img = Image(image_path, width=orig_width/2, height=orig_height/2)  # Adjust size as needed
                elements.append(img)
                elements.append(Spacer(1, 12))



doc.build(elements, onFirstPage = add_page_number, onLaterPages = add_page_number)
print(f'PDF created: {pdf_path}')

In [ ]:
#from IPython.display import FileLink

#FileLink(pdf_path)